In [0]:
import torch
from torch import nn
import torchvision as tv
from torchvision import transforms
import time

In [2]:
train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)

0it [00:00, ?it/s]

26427392it [00:02, 12527555.88it/s]                              


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 94151.55it/s]                            
0it [00:00, ?it/s]

Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw


4423680it [00:01, 4030879.38it/s]                             
0it [00:00, ?it/s]

Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw


8192it [00:00, 30900.85it/s]            

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw
Processing...
Done!


In [0]:
BATCH_SIZE = 256
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Модель и обучение

In [0]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [0]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [0]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [0]:
# Базовая модель от Алексея :)

In [0]:
model = torch.nn.Sequential(
    nn.Flatten(),
    nn.BatchNorm1d(784),
    
    nn.Linear(784, 4096),
    nn.ReLU(),
    nn.BatchNorm1d(4096),
    nn.Dropout(0.1),
    
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.1),
    
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.1),
    
    nn.Linear(512, 10)
).to(dev)

In [0]:
# Пробуем изменить модель в ширину добавив нейронов в слои 

In [0]:
model = torch.nn.Sequential(
    nn.Flatten(),
    nn.BatchNorm1d(784),
    
    nn.Linear(784, 8192),
    nn.ReLU(),
    nn.BatchNorm1d(8192),
    nn.Dropout(0.1),
    
    nn.Linear(8192, 2048),
    nn.ReLU(),
    nn.BatchNorm1d(2048),
    nn.Dropout(0.1),
    
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.1),
    
    nn.Linear(1024, 10)
).to(dev)

In [13]:
lr, num_epochs = 0.005, 20
trainer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.5)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 1.8396, train acc 0.756, test acc 0.811, time 8.4 sec
epoch 2, loss 0.4397, train acc 0.842, test acc 0.843, time 8.2 sec
epoch 3, loss 0.3666, train acc 0.867, test acc 0.859, time 8.4 sec
epoch 4, loss 0.3344, train acc 0.878, test acc 0.863, time 8.3 sec
epoch 5, loss 0.3032, train acc 0.888, test acc 0.855, time 8.3 sec
epoch 6, loss 0.2797, train acc 0.896, test acc 0.876, time 8.3 sec
epoch 7, loss 0.2627, train acc 0.902, test acc 0.881, time 8.2 sec
epoch 8, loss 0.2429, train acc 0.909, test acc 0.885, time 8.2 sec
epoch 9, loss 0.2362, train acc 0.911, test acc 0.880, time 8.2 sec
epoch 10, loss 0.2211, train acc 0.916, test acc 0.882, time 8.1 sec
epoch 11, loss 0.2080, train acc 0.922, test acc 0.882, time 8.0 sec
epoch 12, loss 0.1966, train acc 0.926, test acc 0.882, time 8.0 sec
epoch 13, loss 0.1879, train acc 0.928, test acc 0.888, time 8.1 sec
epoch 14, loss 0.1806, train acc 0.931, test acc 0.882, time 8.0 sec
epoch 15, loss 0.1716, train acc 0.935, tes

In [14]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.0908, train acc 0.966, test acc 0.896, time 8.3 sec
epoch 2, loss 0.0733, train acc 0.973, test acc 0.899, time 8.0 sec
epoch 3, loss 0.0658, train acc 0.976, test acc 0.898, time 7.8 sec
epoch 4, loss 0.0595, train acc 0.978, test acc 0.896, time 7.9 sec
epoch 5, loss 0.0556, train acc 0.980, test acc 0.896, time 7.9 sec
epoch 6, loss 0.0511, train acc 0.982, test acc 0.896, time 8.1 sec
epoch 7, loss 0.0496, train acc 0.982, test acc 0.894, time 8.1 sec
epoch 8, loss 0.0461, train acc 0.983, test acc 0.895, time 8.1 sec
epoch 9, loss 0.0444, train acc 0.984, test acc 0.896, time 8.2 sec
epoch 10, loss 0.0400, train acc 0.985, test acc 0.898, time 8.1 sec


In [15]:
lr, num_epochs = 0.0001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.0356, train acc 0.987, test acc 0.898, time 7.7 sec
epoch 2, loss 0.0341, train acc 0.988, test acc 0.898, time 7.5 sec
epoch 3, loss 0.0328, train acc 0.988, test acc 0.896, time 7.6 sec
epoch 4, loss 0.0321, train acc 0.989, test acc 0.896, time 7.6 sec
epoch 5, loss 0.0304, train acc 0.990, test acc 0.897, time 7.7 sec
epoch 6, loss 0.0306, train acc 0.989, test acc 0.898, time 7.6 sec
epoch 7, loss 0.0307, train acc 0.989, test acc 0.899, time 7.6 sec
epoch 8, loss 0.0302, train acc 0.989, test acc 0.897, time 7.6 sec
epoch 9, loss 0.0308, train acc 0.989, test acc 0.896, time 7.5 sec
epoch 10, loss 0.0291, train acc 0.990, test acc 0.896, time 7.6 sec


In [16]:
lr, num_epochs = 0.00001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.0285, train acc 0.990, test acc 0.897, time 7.9 sec
epoch 2, loss 0.0289, train acc 0.990, test acc 0.900, time 7.5 sec
epoch 3, loss 0.0293, train acc 0.990, test acc 0.897, time 7.6 sec
epoch 4, loss 0.0287, train acc 0.990, test acc 0.898, time 7.5 sec
epoch 5, loss 0.0281, train acc 0.991, test acc 0.897, time 7.5 sec
epoch 6, loss 0.0286, train acc 0.990, test acc 0.898, time 7.6 sec
epoch 7, loss 0.0287, train acc 0.990, test acc 0.896, time 7.5 sec
epoch 8, loss 0.0275, train acc 0.990, test acc 0.898, time 7.4 sec
epoch 9, loss 0.0289, train acc 0.990, test acc 0.898, time 7.4 sec
epoch 10, loss 0.0279, train acc 0.990, test acc 0.898, time 7.3 sec


In [0]:
# Попали примерно в тотже минимум (результаты близки)

In [0]:
# Пробуем изменить модель в глубину добавив еще один слой 

In [0]:
model = torch.nn.Sequential(
    nn.Flatten(),
    nn.BatchNorm1d(784),
    
    nn.Linear(784, 4096),
    nn.ReLU(),
    nn.BatchNorm1d(4096),
    nn.Dropout(0.1),
    
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.BatchNorm1d(4096),
    nn.Dropout(0.1),

    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.1),
    
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.1),
    
    nn.Linear(512, 10)
).to(dev)

In [18]:
lr, num_epochs = 0.005, 20
trainer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.5)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 1.2553, train acc 0.753, test acc 0.808, time 7.7 sec
epoch 2, loss 0.4434, train acc 0.838, test acc 0.841, time 7.7 sec
epoch 3, loss 0.3758, train acc 0.863, test acc 0.854, time 7.8 sec
epoch 4, loss 0.3342, train acc 0.877, test acc 0.856, time 8.0 sec
epoch 5, loss 0.3074, train acc 0.887, test acc 0.867, time 7.8 sec
epoch 6, loss 0.2869, train acc 0.894, test acc 0.874, time 7.9 sec
epoch 7, loss 0.2666, train acc 0.902, test acc 0.880, time 7.9 sec
epoch 8, loss 0.2502, train acc 0.907, test acc 0.874, time 7.9 sec
epoch 9, loss 0.2390, train acc 0.911, test acc 0.875, time 8.0 sec
epoch 10, loss 0.2266, train acc 0.914, test acc 0.884, time 8.3 sec
epoch 11, loss 0.2131, train acc 0.921, test acc 0.885, time 8.2 sec
epoch 12, loss 0.2037, train acc 0.924, test acc 0.882, time 8.2 sec
epoch 13, loss 0.1945, train acc 0.928, test acc 0.881, time 8.2 sec
epoch 14, loss 0.1882, train acc 0.930, test acc 0.886, time 8.3 sec
epoch 15, loss 0.1772, train acc 0.934, tes

In [19]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.0963, train acc 0.965, test acc 0.892, time 8.0 sec
epoch 2, loss 0.0795, train acc 0.971, test acc 0.895, time 7.9 sec
epoch 3, loss 0.0735, train acc 0.974, test acc 0.892, time 7.9 sec
epoch 4, loss 0.0689, train acc 0.975, test acc 0.896, time 7.9 sec
epoch 5, loss 0.0639, train acc 0.977, test acc 0.895, time 7.9 sec
epoch 6, loss 0.0594, train acc 0.979, test acc 0.893, time 7.9 sec
epoch 7, loss 0.0578, train acc 0.979, test acc 0.895, time 8.0 sec
epoch 8, loss 0.0558, train acc 0.980, test acc 0.897, time 7.9 sec
epoch 9, loss 0.0516, train acc 0.981, test acc 0.894, time 7.8 sec
epoch 10, loss 0.0504, train acc 0.982, test acc 0.896, time 7.8 sec


In [20]:
lr, num_epochs = 0.0001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.0449, train acc 0.984, test acc 0.895, time 7.9 sec
epoch 2, loss 0.0422, train acc 0.985, test acc 0.894, time 7.8 sec
epoch 3, loss 0.0429, train acc 0.985, test acc 0.894, time 7.7 sec
epoch 4, loss 0.0413, train acc 0.985, test acc 0.897, time 7.7 sec
epoch 5, loss 0.0417, train acc 0.985, test acc 0.895, time 7.7 sec
epoch 6, loss 0.0407, train acc 0.986, test acc 0.895, time 7.7 sec
epoch 7, loss 0.0404, train acc 0.985, test acc 0.896, time 7.6 sec
epoch 8, loss 0.0394, train acc 0.986, test acc 0.896, time 7.7 sec
epoch 9, loss 0.0382, train acc 0.986, test acc 0.894, time 7.6 sec
epoch 10, loss 0.0390, train acc 0.986, test acc 0.895, time 7.8 sec


In [21]:
lr, num_epochs = 0.00001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.0387, train acc 0.986, test acc 0.897, time 7.8 sec
epoch 2, loss 0.0389, train acc 0.986, test acc 0.893, time 7.7 sec
epoch 3, loss 0.0394, train acc 0.986, test acc 0.894, time 8.0 sec
epoch 4, loss 0.0390, train acc 0.986, test acc 0.897, time 8.2 sec
epoch 5, loss 0.0374, train acc 0.987, test acc 0.895, time 8.2 sec
epoch 6, loss 0.0386, train acc 0.987, test acc 0.893, time 8.1 sec
epoch 7, loss 0.0376, train acc 0.987, test acc 0.895, time 7.8 sec
epoch 8, loss 0.0384, train acc 0.986, test acc 0.894, time 7.7 sec
epoch 9, loss 0.0384, train acc 0.986, test acc 0.897, time 7.8 sec
epoch 10, loss 0.0380, train acc 0.987, test acc 0.897, time 8.1 sec


In [0]:
# Результат чуть хуже, но не значительно

In [0]:
# Изменим модель в глубину добавив еще один слой

In [0]:
model = torch.nn.Sequential(
    nn.Flatten(),
    nn.BatchNorm1d(784),
    
    nn.Linear(784, 4096),
    nn.ReLU(),
    nn.BatchNorm1d(4096),
    nn.Dropout(0.1),
    
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.BatchNorm1d(4096),
    nn.Dropout(0.1),

    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.1),
    
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    
    nn.Linear(128, 10)
).to(dev)

In [0]:
# Изменим оптимизатор на первой фазе обучения на Adam (в надежде, что оптимизатор доведет нас до другого локального минимума)

In [31]:
lr, num_epochs = 0.01, 20
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.4313, train acc 0.847, test acc 0.855, time 9.7 sec
epoch 2, loss 0.3464, train acc 0.874, test acc 0.860, time 9.6 sec
epoch 3, loss 0.3198, train acc 0.883, test acc 0.853, time 9.6 sec
epoch 4, loss 0.3075, train acc 0.887, test acc 0.865, time 9.7 sec
epoch 5, loss 0.2948, train acc 0.890, test acc 0.869, time 9.6 sec
epoch 6, loss 0.2730, train acc 0.899, test acc 0.878, time 9.6 sec
epoch 7, loss 0.2672, train acc 0.901, test acc 0.873, time 9.6 sec
epoch 8, loss 0.2562, train acc 0.903, test acc 0.882, time 9.6 sec
epoch 9, loss 0.2432, train acc 0.909, test acc 0.884, time 9.5 sec
epoch 10, loss 0.2300, train acc 0.913, test acc 0.883, time 9.2 sec
epoch 11, loss 0.2182, train acc 0.919, test acc 0.883, time 9.1 sec
epoch 12, loss 0.2103, train acc 0.921, test acc 0.881, time 9.1 sec
epoch 13, loss 0.2075, train acc 0.922, test acc 0.887, time 9.1 sec
epoch 14, loss 0.2001, train acc 0.926, test acc 0.887, time 9.3 sec
epoch 15, loss 0.1902, train acc 0.929, tes

In [32]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.1200, train acc 0.956, test acc 0.891, time 7.9 sec
epoch 2, loss 0.1150, train acc 0.957, test acc 0.893, time 7.8 sec
epoch 3, loss 0.1140, train acc 0.958, test acc 0.893, time 7.6 sec
epoch 4, loss 0.1106, train acc 0.959, test acc 0.894, time 7.7 sec
epoch 5, loss 0.1098, train acc 0.959, test acc 0.895, time 7.7 sec
epoch 6, loss 0.1089, train acc 0.959, test acc 0.895, time 7.8 sec
epoch 7, loss 0.1070, train acc 0.960, test acc 0.895, time 7.7 sec
epoch 8, loss 0.1077, train acc 0.960, test acc 0.895, time 7.7 sec
epoch 9, loss 0.1062, train acc 0.961, test acc 0.893, time 7.7 sec
epoch 10, loss 0.1062, train acc 0.960, test acc 0.896, time 7.6 sec


In [33]:
lr, num_epochs = 0.0001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.1052, train acc 0.961, test acc 0.895, time 7.8 sec
epoch 2, loss 0.1051, train acc 0.960, test acc 0.895, time 7.6 sec
epoch 3, loss 0.1049, train acc 0.962, test acc 0.899, time 7.6 sec
epoch 4, loss 0.1052, train acc 0.961, test acc 0.896, time 7.6 sec
epoch 5, loss 0.1051, train acc 0.961, test acc 0.897, time 7.6 sec
epoch 6, loss 0.1046, train acc 0.961, test acc 0.894, time 7.7 sec
epoch 7, loss 0.1045, train acc 0.961, test acc 0.895, time 7.8 sec
epoch 8, loss 0.1038, train acc 0.962, test acc 0.896, time 7.6 sec
epoch 9, loss 0.1055, train acc 0.961, test acc 0.894, time 7.6 sec
epoch 10, loss 0.1032, train acc 0.962, test acc 0.896, time 7.7 sec


In [34]:
lr, num_epochs = 0.00001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.1050, train acc 0.961, test acc 0.895, time 7.7 sec
epoch 2, loss 0.1043, train acc 0.961, test acc 0.894, time 7.6 sec
epoch 3, loss 0.1044, train acc 0.961, test acc 0.895, time 7.6 sec
epoch 4, loss 0.1043, train acc 0.962, test acc 0.897, time 7.6 sec
epoch 5, loss 0.1036, train acc 0.962, test acc 0.896, time 7.8 sec
epoch 6, loss 0.1033, train acc 0.962, test acc 0.897, time 7.6 sec
epoch 7, loss 0.1047, train acc 0.961, test acc 0.895, time 7.6 sec
epoch 8, loss 0.1056, train acc 0.961, test acc 0.897, time 7.8 sec
epoch 9, loss 0.1026, train acc 0.962, test acc 0.894, time 8.0 sec
epoch 10, loss 0.1034, train acc 0.962, test acc 0.896, time 8.0 sec


In [0]:
# Получили результат хуже чем при других оптимизаторах, 
# однако модель все таки сошлась в другой локальный мимнимум

In [0]:
# Попробуем простую модель

In [0]:
model = torch.nn.Sequential(
    nn.Flatten(),
    nn.BatchNorm1d(784),
    
    nn.Linear(784, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.1),
    
    
    nn.Linear(512, 10),
).to(dev)

In [36]:
lr, num_epochs = 0.01, 20
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.4989, train acc 0.822, test acc 0.838, time 5.9 sec
epoch 2, loss 0.3682, train acc 0.865, test acc 0.861, time 5.9 sec
epoch 3, loss 0.3246, train acc 0.879, test acc 0.865, time 5.9 sec
epoch 4, loss 0.3007, train acc 0.889, test acc 0.863, time 5.9 sec
epoch 5, loss 0.2832, train acc 0.894, test acc 0.873, time 5.9 sec
epoch 6, loss 0.2681, train acc 0.900, test acc 0.877, time 5.9 sec
epoch 7, loss 0.2585, train acc 0.903, test acc 0.877, time 5.9 sec
epoch 8, loss 0.2476, train acc 0.907, test acc 0.872, time 6.0 sec
epoch 9, loss 0.2381, train acc 0.910, test acc 0.876, time 5.9 sec
epoch 10, loss 0.2312, train acc 0.912, test acc 0.872, time 5.9 sec
epoch 11, loss 0.2242, train acc 0.917, test acc 0.882, time 5.9 sec
epoch 12, loss 0.2218, train acc 0.917, test acc 0.880, time 5.9 sec
epoch 13, loss 0.2100, train acc 0.920, test acc 0.881, time 5.9 sec
epoch 14, loss 0.1979, train acc 0.926, test acc 0.875, time 5.9 sec
epoch 15, loss 0.1950, train acc 0.927, tes

In [37]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.1366, train acc 0.949, test acc 0.891, time 5.7 sec
epoch 2, loss 0.1263, train acc 0.954, test acc 0.893, time 5.6 sec
epoch 3, loss 0.1221, train acc 0.956, test acc 0.895, time 5.7 sec
epoch 4, loss 0.1196, train acc 0.956, test acc 0.895, time 5.6 sec
epoch 5, loss 0.1167, train acc 0.957, test acc 0.895, time 5.7 sec
epoch 6, loss 0.1135, train acc 0.958, test acc 0.896, time 5.7 sec
epoch 7, loss 0.1121, train acc 0.958, test acc 0.894, time 5.7 sec
epoch 8, loss 0.1116, train acc 0.958, test acc 0.893, time 5.6 sec
epoch 9, loss 0.1095, train acc 0.959, test acc 0.894, time 5.6 sec
epoch 10, loss 0.1089, train acc 0.959, test acc 0.896, time 5.1 sec


In [38]:
lr, num_epochs = 0.0001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.1063, train acc 0.961, test acc 0.895, time 5.2 sec
epoch 2, loss 0.1058, train acc 0.961, test acc 0.895, time 5.1 sec
epoch 3, loss 0.1055, train acc 0.961, test acc 0.896, time 5.2 sec
epoch 4, loss 0.1061, train acc 0.961, test acc 0.896, time 5.3 sec
epoch 5, loss 0.1048, train acc 0.961, test acc 0.895, time 5.2 sec
epoch 6, loss 0.1034, train acc 0.962, test acc 0.895, time 5.2 sec
epoch 7, loss 0.1047, train acc 0.962, test acc 0.894, time 5.4 sec
epoch 8, loss 0.1048, train acc 0.962, test acc 0.894, time 5.5 sec
epoch 9, loss 0.1041, train acc 0.961, test acc 0.898, time 5.5 sec
epoch 10, loss 0.1037, train acc 0.961, test acc 0.895, time 5.3 sec


In [39]:
lr, num_epochs = 0.00001, 10
trainer = torch.optim.SGD(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

epoch 1, loss 0.1035, train acc 0.962, test acc 0.894, time 5.3 sec
epoch 2, loss 0.1050, train acc 0.961, test acc 0.896, time 5.1 sec
epoch 3, loss 0.1040, train acc 0.962, test acc 0.897, time 5.3 sec
epoch 4, loss 0.1035, train acc 0.962, test acc 0.896, time 5.3 sec
epoch 5, loss 0.1032, train acc 0.962, test acc 0.895, time 5.1 sec
epoch 6, loss 0.1046, train acc 0.961, test acc 0.895, time 5.6 sec
epoch 7, loss 0.1038, train acc 0.962, test acc 0.896, time 5.9 sec
epoch 8, loss 0.1043, train acc 0.961, test acc 0.895, time 5.8 sec
epoch 9, loss 0.1044, train acc 0.962, test acc 0.895, time 5.7 sec
epoch 10, loss 0.1037, train acc 0.962, test acc 0.895, time 5.6 sec


In [0]:
# Видно, что падения качества не произошло, модель сошлась примерно в тот же минимум, что и предыдущая

In [0]:
# получить качество в 92% не получилось :(